# Usando SentiStrength

## Api do sentistrength

In [31]:
import requests 
import re
import json
import pandas as pd

In [23]:
def arrumaQuery(query):
    ''' funcao auxiliar para arrumar a query'''
    return query.replace(' ', '+').replace('\n', '+')

In [27]:
def retornaSentimentoComSentiStrength(query):
    '''funcao que avalia os sentimentos utilizando 
    o SentiStrength retornando os 
    resultados posivitos e negativos'''
    BaseUrl =  "http://sentistrength.wlv.ac.uk"

    DOMAIN = "AutoDetect";
    SUBMIT = "Detect+Sentiment+in+Domain"
    
    # monta URL
    url = BaseUrl+'/results.php?domain='+DOMAIN+'&submit='+SUBMIT+'&text='+arrumaQuery(query)
    
#     print(url)
    #requisita
    r = requests.get(url = url)
    
    # usa regex para capturar nota
    notaExpr = re.compile(r'has positive strength <b>.*</b> and negative strength <b>.*</b>')
    textoFiltrado = re.findall(notaExpr, str(r.content))
    pegaNotaPositiva = str(textoFiltrado).split('<b>')[1].split('</b>')[0]
    pegaNotaNegativa = str(textoFiltrado).split('<b>')[2].split('</b>')[0]
    
    return {'NotaPositiva':int(pegaNotaPositiva), 'NotaNegativa':int(pegaNotaNegativa)}

In [28]:
retornaSentimentoComSentiStrength('this is a horrible idea')

{'NotaPositiva': 1, 'NotaNegativa': -4}

In [49]:
dados = []
# pd.DataFrame(columns=['autor', 'data', 'mensagem', 'NotaPos', 'NotaNeg'])
with open('exemplo.json', 'r') as arq:
    for commit in json.loads(arq.read()):
#         print(commit)
        nota = retornaSentimentoComSentiStrength(commit['mensagem'])
#         print('autor:', commit['autor'])
#         print('data:', commit['data'])
#         print('mensagem:', commit['mensagem'])
#         print('sentiment', nota)
        dados.append([
            commit['autor'],
            commit['data'],
            commit['mensagem'],
            nota['NotaPositiva'],
            nota['NotaNegativa']            
        ])

In [60]:
dados = pd.DataFrame(dados, columns=['autor', 'data', 'mensagem', 'NotaPos', 'NotaNeg']).set_index('autor')

In [65]:
dados.sort_index()[['NotaPos','NotaNeg', 'mensagem', 'data']].to_csv('saida_Codigo.csv')